# Renda per Capita aproximada por setor censitário

Exemplo de aplicação com os dados do IBGE

In [4]:
import os
import pandas as pd

# 1. Carregando a base que relaciona CEP, coordenadas geográficas e código do setor censitário

In [3]:
census_code_cep_coordinates = pd.read_csv("../data/processed/census_code_cep_coordinates.csv")
census_code_cep_coordinates.head()

,POSTCODE,LON,LAT,CD_GEOCODI
0,00000-000,-46.663830,-22.257613,313490505000032
1,00000-001,-67.838738,-9.921660,120040105000325
2,00000-002,-67.880508,-9.882072,120040105000325
3,00000-003,-46.514443,-21.820570,315180005000240
4,00000-004,-67.893353,-9.882038,120040105000325


# 2. Concatenando os arquivos PessoaRenda_UF.csv

In [5]:
pessoa_renda_paths = []
for dirname, dirnames, filenames in os.walk('../data/agregados/extracted/'):
    # get all pessoarenda paths
    for filename in filenames:
        if ("PessoaRenda" in filename) and (".csv" in filename):
            pessoa_renda_paths.append(os.path.join(dirname, filename))

In [6]:
pessoa_renda_paths[:3]

['../data/agregados/extracted/SE/Base informaçoes setores2010 universo SE/CSV/PessoaRenda_SE.csv',
 '../data/agregados/extracted/PE_20171016/PE/Base informaçoes setores2010 universo PE/CSV/PessoaRenda_PE.csv',
 '../data/agregados/extracted/SC/Base informaçoes setores2010 universo SC/CSV/PessoaRenda_SC.csv']

Da documentação:

- V020: Pessoas de 10 anos ou mais de idade com ou sem rendimento
- V022: Total do rendimento nominal mensal das pessoas de 10 anos ou mais de idade

In [8]:
dfs = [pd.read_csv(f"{file_path}", sep=";", usecols=["Cod_setor", "V020", "V022"]) for file_path in pessoa_renda_paths]
pessoa_renda_df = pd.concat(dfs)
del dfs
pessoa_renda_df.head()

,Cod_setor,V020,V022
0,280010005000001,727,263997
1,280010005000002,378,178925
2,280010005000003,395,91417
3,280010005000004,296,62218
4,280010005000005,51,11121


In [9]:
# cleaning wrong data
pessoa_renda_df = pessoa_renda_df[pessoa_renda_df["V020"]!="X"]

In [10]:
pessoa_renda_df["V020"] = pessoa_renda_df["V020"].astype(int)
pessoa_renda_df["V022"] = pessoa_renda_df["V022"].astype(int)

# 3. Calculando a renda per capita aproximada

In [11]:
pessoa_renda_df["renda_per_capita"] = pessoa_renda_df["V022"] / pessoa_renda_df["V020"]

In [12]:
pessoa_renda_df = pessoa_renda_df[["Cod_setor", "renda_per_capita"]]

In [13]:
pessoa_renda_df = pessoa_renda_df.rename(columns={
    "Cod_setor": "CD_GEOCODI"
})

In [14]:
cep_coordinates_per_capita_income = census_code_cep_coordinates.merge(pessoa_renda_df, on="CD_GEOCODI")

In [17]:
cep_coordinates_per_capita_income.sample(10)

,POSTCODE,LON,LAT,CD_GEOCODI,renda_per_capita
135261,26585-610,-43.444497,-22.805094,330285805000177,451.929603
102894,21540-040,-43.342976,-22.863863,330455705200244,624.132388
121526,25223-505,-43.258575,-22.694976,330170210000202,552.155529
193190,37716-102,-46.583734,-21.795178,315180005000233,3538.013559
184309,35702-112,-44.240988,-19.494802,316720205150009,670.161202
389712,88095-573,-48.603475,-27.588623,420540705000340,835.044892
111195,23068-300,-43.603791,-22.881477,330455705230190,330.989091
116606,24415-330,-43.071071,-22.839505,330490420000185,1059.356299
3800,01547-060,-46.621222,-23.581915,355030890000147,1761.898148
340134,78098-682,-55.981633,-15.650340,510340310430107,729.680605


# 4. Salvando o arquivo

In [ ]:
cep_coordinates_per_capita_income.to_csv("../data/processed/cep_coordinates_per_capita_income.csv", index=False)

# 5. Exemplo de gráfico

Fonte: [https://www.kaggle.com/silveiragustavo/geospatial-analysis-income-data](https://www.kaggle.com/silveiragustavo/geospatial-analysis-income-data)

![geospatial_analysis](../geospatial_plot.png)